# --- Day 11: Dumbo Octopus ---
https://adventofcode.com/2021/day/11

You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.

There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.

Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input).

The energy level of each octopus is a value between 0 and 9.

You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

- First, the energy level of each octopus increases by 1.
- Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
- Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.

Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

In [2]:
def octopusLocations():
    with open('dumboOctopuses.txt') as file:
        return file.read()
    
def octopusLocationsTest():
    with open('dumboOctopusesTest.txt') as file:
        return file.read()
    
print(octopusLocations())

4721224663
6875415276
2742448428
4878231556
5684643743
3553681866
4788183625
4255856532
1415818775
2326886125


In [11]:
#formatting input
dumboOct = octopusLocations().split('\n')
for i, val in enumerate(dumboOct): #Turns rows into lists of items in the row
    dumboOct[i] = list(val)

def getSurrounding(octopuses, row, col):
    """This takes in the map of octopuses, and then the coordinates of the octopus it needs to find
    the surrounding values of the octopus"""
    up = None
    down = None
    left = None
    right = None
    upLeft = None
    upRight = None
    downRight = None
    downLeft = None
    if abs(row-1) == row-1: #Gets the value of the energy above octopuses[row][col] if it exists else None
        up = octopuses[row-1][col]
    if row+1 < len(octopuses): #Gets the value of the energy below octopuses[row][col] if it exists else None
        down = octopuses[row+1][col]
    if abs(col-1) == col-1: #Gets the value of the energy left of octopuses[row][col] if it exists else None
        left = octopuses[row][col-1]
    if col+1 < len(octopuses[row]): #Gets the value of the energy right of octopuses[row][col] if it exists else None
        right = octopuses[row][col+1]
    if abs(col-1) == col-1 and abs(row-1) == row-1: #Gets upper left value
        upLeft = octopuses[row-1][col-1]
    if col+1 < len(octopuses[row]) and abs(row-1) == row-1: #Gets upper right value
        upRight = octopuses[row-1][col+1]
    if col+1 < len(octopuses[row]) and row+1 < len(octopuses): #Gets lower right value
        downRight = octopuses[row+1][col+1]
    if abs(col-1) == col-1 and row+1 < len(octopuses): #Gets lower left value
        downLeft = octopuses[row+1][col-1]
    return up, down, left, right, upLeft, upRight, downRight, downLeft
    
class dumboOctopus:
    allOctopuses = []
    flashes = 0
    def __init__(self, energy):
        self.energy = energy
        self.reset = False
    
    @classmethod
    def recurseOct(cls, row, col):
        cls.allOctopuses[row][col].reset=True
        cls.allOctopuses[row][col].energy = 0
        up, down, left, right, upLeft, upRight, downRight, downLeft = getSurrounding(cls.allOctopuses, row, col)
        if up and not up.reset: #if up exists and has not already reset
            up.energy+=1
            if up>9:
                cls.recurseOct(row-1, col)
        if down and not down.reset: #down
            down.energy+=1
            if down>9:
                cls.recurseOct(row+1, col)
        if left and not left.reset: #left
            left.energy+=1
            if left>9:
                cls.recurseOct(row, col-1)
        if right and not right.reset: #right
            right.energy+=1
            if right>9:
                cls.recurseOct(row, col+1)
        if upLeft and not upLeft.reset: #upLeft
            upLeft.energy+=1
            if upLeft>9:
                cls.recurseOct(row-1, col-1)
        if upRight and not upRight.reset: #upRight
            upRight.energy+=1
            if upRight>9:
                cls.recurseOct(row-1, col+1)
        if downRight and not downRight.reset: #downRight
            downRight.energy+=1
            if downRight>9:
                cls.recurseOct(row+1, col+1)
        if downLeft and not downLeft.reset: #downLeft
            downLeft.energy+=1
            if downLeft>9:
                cls.recurseOct(row+1, col-1)
    
    @classmethod
    def step(cls):
        """This simulates one step in the octopus map for each octopus"""
        for row in range(len(cls.allOctopuses)):
            for col in range(len(cls.allOctopuses[row])):
                if not cls.allOctopuses[row][col].reset: #If octopus has not reset yet already
                    cls.allOctopuses[row][col].energy+=1
                    if cls.allOctopuses[row][col].energy>9: #Only when octopus energy resets
                        cls.recurseOct(row, col)
        
        #This checks to see if each octopus has reset or not. If it has then it adds one flash and sets reset to False
        for row in range(len(cls.allOctopuses)):
            for col in range(len(cls.allOctopuses[row])):
                if cls.allOctopuses[row][col].reset:
                    cls.flashes+=1
                    cls.allOctopuses[row][col].reset = False
    
    def __gt__(self, other):
        """Returns a boolean value depending on weather the energy level is greater than other"""
        return self.energy > other
        
for i in range(len(dumboOct)): #Casts all values as dumboOctopus instances
    for j in range(len(dumboOct[i])):
        dumboOct[i][j] = dumboOctopus(int(dumboOct[i][j]))
dumboOctopus.allOctopuses = dumboOct #Sets class variable allOctopuses to all instances of the class in the correct spot

steps = 100
for i in range(steps):
    dumboOctopus.step()
print(f"Total number of flashes: {dumboOctopus.flashes}")

Total number of flashes: 1675


# --- Part Two ---
https://adventofcode.com/2021/day/11#part2

It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be synchronizing!

In the example above, the first time all octopuses flash simultaneously is step 195

If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. What is the first step during which all octopuses flash?

In [109]:
#formatting input
dumboOct = octopusLocations().split('\n')
for i, val in enumerate(dumboOct): #Turns rows into lists of items in the row
    dumboOct[i] = list(val)

def getSurrounding(octopuses, row, col):
    """This takes in the map of octopuses, and then the coordinates of the octopus it needs to find
    the surrounding values of the octopus"""
    up = None
    down = None
    left = None
    right = None
    upLeft = None
    upRight = None
    downRight = None
    downLeft = None
    if abs(row-1) == row-1: #Gets the value of the energy above octopuses[row][col] if it exists else None
        up = octopuses[row-1][col]
    if row+1 < len(octopuses): #Gets the value of the energy below octopuses[row][col] if it exists else None
        down = octopuses[row+1][col]
    if abs(col-1) == col-1: #Gets the value of the energy left of octopuses[row][col] if it exists else None
        left = octopuses[row][col-1]
    if col+1 < len(octopuses[row]): #Gets the value of the energy right of octopuses[row][col] if it exists else None
        right = octopuses[row][col+1]
    if abs(col-1) == col-1 and abs(row-1) == row-1: #Gets upper left value
        upLeft = octopuses[row-1][col-1]
    if col+1 < len(octopuses[row]) and abs(row-1) == row-1: #Gets upper right value
        upRight = octopuses[row-1][col+1]
    if col+1 < len(octopuses[row]) and row+1 < len(octopuses): #Gets lower right value
        downRight = octopuses[row+1][col+1]
    if abs(col-1) == col-1 and row+1 < len(octopuses): #Gets lower left value
        downLeft = octopuses[row+1][col-1]
    return up, down, left, right, upLeft, upRight, downRight, downLeft
    
class dumboOctopus:
    allOctopuses = []#Keeps track of all octopuses in the
    flashes=0
    currentStep=0
    
    def __init__(self, energy):
        self.energy = energy
        self.reset = False
    
    @classmethod
    def recurseOct(cls, row, col):
        cls.allOctopuses[row][col].reset=True
        cls.allOctopuses[row][col].energy = 0
        up, down, left, right, upLeft, upRight, downRight, downLeft = getSurrounding(cls.allOctopuses, row, col)
        if up and not up.reset: #if up exists and has not already reset
            up.energy+=1
            if up>9:
                cls.recurseOct(row-1, col)
        if down and not down.reset: #down
            down.energy+=1
            if down>9:
                cls.recurseOct(row+1, col)
        if left and not left.reset: #left
            left.energy+=1
            if left>9:
                cls.recurseOct(row, col-1)
        if right and not right.reset: #right
            right.energy+=1
            if right>9:
                cls.recurseOct(row, col+1)
        if upLeft and not upLeft.reset: #upLeft
            upLeft.energy+=1
            if upLeft>9:
                cls.recurseOct(row-1, col-1)
        if upRight and not upRight.reset: #upRight
            upRight.energy+=1
            if upRight>9:
                cls.recurseOct(row-1, col+1)
        if downRight and not downRight.reset: #downRight
            downRight.energy+=1
            if downRight>9:
                cls.recurseOct(row+1, col+1)
        if downLeft and not downLeft.reset: #downLeft
            downLeft.energy+=1
            if downLeft>9:
                cls.recurseOct(row+1, col-1)
                
    @classmethod
    def allReset(cls):
        """Checks to see if all octopus have reset (happens when all octopuses are in sync)"""
        for row in range(len(cls.allOctopuses)):
            for col in range(len(cls.allOctopuses[row])):
                if cls.allOctopuses[row][col].energy != 0: #If energy is not 0 then they are not all synchronized
                    return False
        return True #If it hasn't already returned then all octopuses are synchronized
    
    @classmethod
    def step(cls):
        """This simulates one step in the octopus map for each octopus"""
        for row in range(len(cls.allOctopuses)):
            for col in range(len(cls.allOctopuses[row])):
                if not cls.allOctopuses[row][col].reset: #If octopus has not reset yet already
                    cls.allOctopuses[row][col].energy+=1
                    if cls.allOctopuses[row][col].energy>9: #Only when octopus energy resets
                        cls.recurseOct(row, col)
        
        #This checks to see if each octopus has reset or not. If it has then it adds one flash and sets reset to False
        for row in range(len(cls.allOctopuses)):
            for col in range(len(cls.allOctopuses[row])):
                if cls.allOctopuses[row][col].reset:
                    cls.flashes+=1
                    cls.allOctopuses[row][col].reset = False
        cls.currentStep+=1
    
    def __gt__(self, other):
        """Returns a boolean value depending on weather the energy level is greater than other"""
        return self.energy > other
    
    def __eq__(self, other):
        """Returns a boolean value depending on weather the energy level is equal to other"""
        return self.energy == other
    
    def __str__(self):
        """Will only return self.energy for testing purposes"""
        return str(self.energy)
        
for i in range(len(dumboOct)): #Casts all values as dumboOctopus instances
    for j in range(len(dumboOct[i])):
        dumboOct[i][j] = dumboOctopus(int(dumboOct[i][j]))
dumboOctopus.allOctopuses = dumboOct #Sets class variable allOctopuses to all instances of the class in the correct spot

while True: #Loops through untill all octopuses are synchronized
    dumboOctopus.step() #Simulates one step
    if dumboOctopus.allReset(): #Checks to see if all octopuses are synchronized
        break

print(f"Total number of flashes: {dumboOctopus.flashes}")
print(f"Number of steps: {dumboOctopus.currentStep}")

Total number of flashes: 8207
Number of steps: 515
